In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 htt

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [13]:
# Filter the dataframe for total_votes greater than 20
vine_df_update = vine_df.filter(vine_df.total_votes > 20)
vine_df_update.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|                Y|
|R1ZPBQCT4DYE4Z|          1|           45|         51|   N|     

In [15]:
# Filter the dataframe for rows with number of helpful_votes / total_votes >= 50%
vine_df_update_2 = vine_df_update.filter(vine_df_update.helpful_votes / vine_df_update.total_votes >= 0.5)
vine_df_update_2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|                Y|
|R1ZPBQCT4DYE4Z|          1|           45|         51|   N|     

In [17]:
# Filter the dataframe for rows where a review was written as part of the Vine program
vine_df_paid = vine_df_update_2.filter(vine_df_update_2.vine == 'Y')
vine_df_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|
|R1803HITBY7Y08|          4|           92|        104|   Y|                N|
|R1888B8Z6QWYZ4|          4|           18|         25|   Y|                N|
|R1S538NK2I5PRL|          5|           21|         22|   Y|                N|
| RZDTBIS1289HJ|          5|           26|         28|   Y|     

In [18]:
# Filter the dataframe for rows where a review was written as part of the Vine program
vine_df_unpaid = vine_df_update_2.filter(vine_df_update_2.vine == 'N')
vine_df_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|                Y|
|R1ZPBQCT4DYE4Z|          1|           45|         51|   N|     

In [39]:
# Determine the total number of reviews, the number of 5 star reviews & percentage of 5-star reviews for the two types of reviews (paid vs. unpaid)
# vine_df_paid.describe(['total_votes', 'star_rating']).show()
# vine_df_unpaid.describe(['total_votes', 'star_rating']).show()

paid_five_star = vine_df_paid.filter(vine_df_paid.star_rating == 5).count()
unpaid_five_star = vine_df_unpaid.filter(vine_df_paid.star_rating == 5).count()

paid_total_reviews = vine_df_paid.count()
unpaid_total_reviews = vine_df_unpaid.count()

paid_five_star_percentage = paid_five_star / paid_total_reviews
unpaid_five_star_percentage = unpaid_five_star / unpaid_total_reviews


0.5319226496319955

In [49]:
print('Total five star ratings for paid: ' + str(paid_five_star))
print('Total five star ratings for unpaid: ' + str(unpaid_five_star))

print('Total reviews for paid: ' + str(paid_total_reviews))
print('Total reviews for unpaid: ' + str(unpaid_total_reviews))

print('Percentage of five star ratings for paid: ' + str(paid_five_star_percentage))
print('Percentage of five star ratings for unpaid: ' + str(unpaid_five_star_percentage))

Total five star ratings for paid: 129
Total five star ratings for unpaid: 30643
Total reviews for paid: 312
Total reviews for unpaid: 57608
Percentage of five star ratings for paid: 0.41346153846153844
Percentage of five star ratings for unpaid: 0.5319226496319955
